In [1]:
import os
import pandas as pd
import openai
import numpy as np
from tqdm import tqdm
from dotenv import load_dotenv
import tiktoken

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']


In [2]:
# 載入TCFD第四層揭露指引 (定義標準 labels)
excel_path = "data/tcfd第四層揭露指引.xlsx"
df_labels = pd.read_excel(excel_path).dropna(subset=['Label', 'Definition'])
label_definitions = df_labels[['Label', 'Definition']].values.tolist()


In [3]:
MODEL_NAME = "gpt-4o-mini"
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

def query_llm_for_verification(chunk, matched_label, label_definition):
    prompt = f"""
    你是氣候財務揭露的專家，請判斷以下報告書內容是否符合這個揭露標準：

    標準代碼：{matched_label}
    標準定義：{label_definition}

    報告內容：
    {chunk}

    ### 回覆格式 ###
    如果符合這個標準，回覆 "Yes"。
    如果不符合，請回覆 "No"，並解釋原因，並說明實際上較符合哪個標準，若都不符合則回覆 "None"。
    """

    try:
        response = openai.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "你是氣候相關財務揭露標準專家。"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error querying LLM: {e}")
        return "Error"


In [4]:
similarity_matched_dir = "data/tcfd_report_pdf_chunks_matching_result_第四層/"
chunk_csv_files = [os.path.join(similarity_matched_dir, f) for f in os.listdir(similarity_matched_dir) if f.endswith('.csv')]

df_chunks_all = pd.concat([pd.read_csv(file) for file in chunk_csv_files], ignore_index=True)
df_chunks_all.head()


,Filename,Chunk_ID,Chunk_Text,Embedding,Matched_Categories,Cosine_Distance
0,上海商銀_2022.pdf,0,2022 年氣候相關財務揭露TCFD 報告書 \n \n-4-\n1 \n治理 \n1.1 ...,"[-0.0016959677450358868, -0.029846400022506714...","['G-2-1_6', 'G-2-1_7', 'G-1-2_3', 'G-2-2_9', '...","[0.10997852683067322, 0.11198242753744125, 0.1..."
1,上海商銀_2022.pdf,1,會，其中由風險管理處主導的環境永續小組，負責評估氣候變遷相關的風險、提出應對策略、推動氣候行...,"[0.004301038105040789, -0.02905302494764328, -...","['G-2-4_12', 'G-1-3_6', 'G-2-1_6', 'G-2-1_7', ...","[0.06479311734437943, 0.07524003833532333, 0.0..."
2,上海商銀_2022.pdf,2,檢視本行氣候行動目標與績效 \n執行關於環境、社會、治理ESG 目標向董事\n會報告氣候策略...,"[-0.011314043775200844, -0.021306172013282776,...","['G-2-4_12', 'G-2-1_6', 'G-2-1_7', 'G-1-2_3', ...","[0.10083988308906555, 0.10233944654464722, 0.1..."
3,上海商銀_2022.pdf,3,-5-\n風險管理委員會 \n風險管理委員會由董事及獨立董事等成員組成，秉於董事會授權，以善...,"[-0.006103315856307745, -0.014614122919738293,...","['G-2-4_12', 'G-1-3_6', 'G-2-1_6', 'G-2-1_7', ...","[0.08129891753196716, 0.0816182941198349, 0.08..."
4,上海商銀_2022.pdf,4,措施。 \n工作小組 \n工作職掌 \n永續發展委員會 \n負責審議氣候相關議題之策略、風險...,"[-0.022168325260281563, -0.021073754876852036,...","['G-2-4_12', 'G-2-1_6', 'G-1-2_3', 'G-1-3_5', ...","[0.09030324220657349, 0.10328415036201477, 0.1..."


In [5]:
verified_output_data = []

for _, row in tqdm(df_chunks_all.iterrows(), total=df_chunks_all.shape[0]):
    file_name = row['Filename']
    chunk_id = row['Chunk_ID']
    chunk_text = row.get('Chunk_Text', "")  # 如果之前的csv沒有Chunk_Text，這裡須再讀取chunk文本檔案

    matched_labels = eval(row['Matched_Categories']) if isinstance(row['Matched_Categories'], str) else row['Matched_Categories']

    for label in matched_labels:
        label_definition = dict(label_definitions).get(label, "No definition found")
        llm_response = query_llm_for_verification(chunk_text, label, label_definition)

        verified_output_data.append({
            "Report": file_name,
            "Chunk_ID": chunk_id,
            "Original_Label": label,
            "LLM_Response": llm_response
        })


100%|██████████| 6883/6883 [5:52:48<00:00,  3.08s/it]   


In [6]:
verified_df = pd.DataFrame(verified_output_data)
verified_df.to_csv("data/llm_question_answering_results/tcfd_gpt_verified_results.csv", index=False)
verified_df.head()


,Report,Chunk_ID,Original_Label,LLM_Response
0,上海商銀_2022.pdf,0,G-2-1_6,No，報告中未明確描述將氣候相關責任分派給特定的管理職位或委員會。雖然提到董事會和永續發展委...
1,上海商銀_2022.pdf,0,G-2-1_7,No，該報告內容沒有明確描述管理職位或委員會是如何向董事會或其下設委員會進行報告的流程或頻率...
2,上海商銀_2022.pdf,0,G-1-2_3,Yes
3,上海商銀_2022.pdf,0,G-2-2_9,Yes
4,上海商銀_2022.pdf,0,G-1-1_1,No，報告中未描述公司是否定期向董事會和/或董事會下設委員會報告氣候相關風險與機會之流程。雖...


In [7]:
# 解析 LLM 回應，判斷實際最終 Label
def parse_llm_response(row):
    response = row['LLM_Response']
    if response.startswith("Yes"):
        return row['Original_Label']
    elif "None" in response:
        return None
    else:
        # 如果LLM建議其他標準，嘗試提取
        possible_labels = [label for label, _ in label_definitions if label in response]
        return possible_labels[0] if possible_labels else None

verified_df['Final_Label'] = verified_df.apply(parse_llm_response, axis=1)
verified_df.head()


,Report,Chunk_ID,Original_Label,LLM_Response,Final_Label
0,上海商銀_2022.pdf,0,G-2-1_6,No，報告中未明確描述將氣候相關責任分派給特定的管理職位或委員會。雖然提到董事會和永續發展委...,G-2-1_6
1,上海商銀_2022.pdf,0,G-2-1_7,No，該報告內容沒有明確描述管理職位或委員會是如何向董事會或其下設委員會進行報告的流程或頻率...,G-2-1_6
2,上海商銀_2022.pdf,0,G-1-2_3,Yes,G-1-2_3
3,上海商銀_2022.pdf,0,G-2-2_9,Yes,G-2-2_9
4,上海商銀_2022.pdf,0,G-1-1_1,No，報告中未描述公司是否定期向董事會和/或董事會下設委員會報告氣候相關風險與機會之流程。雖...,G-1-1_1


In [8]:
verified_df[['Institution', 'Year']] = verified_df['Report'].str.extract(r'(.+?)_(\d{4})')
df_final = verified_df.dropna(subset=['Final_Label'])[['Institution', 'Year', 'Final_Label']].drop_duplicates()
df_final['Answer'] = "Y"

# pivot 成便於比對的格式
df_pivot_llm = df_final.pivot_table(index=['Institution', 'Year'], columns='Final_Label', values='Answer', aggfunc='first').reset_index()
df_pivot_llm.columns.name = None
df_pivot_llm.fillna("N", inplace=True)
df_pivot_llm.head()


,Institution,Year,#MT-1-6_#MT1,#MT-1-7_#MT2,#MT-1-8_#MT3,#MT-1-8_#MT4,#MT-2-4_#MT5,#R-1-4_#R1,#R-1-5_#R2,#S-1-5_#S1,...,S-2-6_36,S-2-6_37,S-3-1_38,S-3-1_39,S-3-2_40,S-3-2_41,S-3-2_42,S-3-2_43,S-3-3_44,S-3-3_45
0,上海商銀,2022,Y,N,N,N,Y,Y,Y,Y,...,N,Y,Y,N,Y,Y,Y,Y,Y,Y
1,中信金控,2021,N,N,N,N,N,N,N,N,...,N,N,Y,Y,N,N,N,Y,Y,Y
2,中信金控,2022,Y,N,Y,N,Y,Y,Y,Y,...,Y,N,Y,Y,Y,Y,Y,Y,Y,Y
3,中信銀行,2022,Y,N,Y,N,Y,Y,Y,Y,...,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
4,中輸銀行,2022,Y,N,N,N,N,Y,Y,Y,...,N,Y,N,N,Y,Y,Y,Y,N,N


In [9]:
answer_file_path = "data/answer/rank.xlsx"
answer_df = pd.read_excel(answer_file_path)
answer_df.columns = answer_df.columns.astype(str)
df_pivot_llm.columns = df_pivot_llm.columns.astype(str)
answer_lookup = { (row['Financial_Institutions'], str(row['Year'])): row for _, row in answer_df.iterrows() }


In [11]:
correct, total = 0, 0

common_cols = [col for col in df_pivot_llm.columns if col not in ['Institution', 'Year'] and col in answer_df.columns]

for _, row in df_pivot_llm.iterrows():
    institution = row['Institution']
    if institution[2:4] == "金控":
        institution = institution[:2] + "金"

    key = (institution, row['Year'])
    answer_row = answer_lookup.get(key)

    if answer_row is None:  # <-- 明確檢查 None
        print(f"No matching data for {key}")
        continue

    for col in common_cols:
        total += 1
        if row[col] == answer_row[col]:
            correct += 1

accuracy = correct / total if total else 0
print(f"總題數: {total}, 正確題數: {correct}, 準確率: {accuracy:.4f}")


No matching data for ('台灣企銀', '2021')
No matching data for ('台灣企銀', '2022')
總題數: 0, 正確題數: 0, 準確率: 0.0000
